VISTO LEZIONE 8 (le prime righe) la seconda parte alla fine della lez 9

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')


In [2]:
from random import randint, random

#testFile(path) does the same as parallelize, but instead of doing it in memory it does it in a file.

labelList = {0:"Milano",1:"Roma",2:"Londra",3:"Tokyio",4:"New York"}
myRdd = sc.parallelize(range(1000)).map(lambda x: {"id":str(randint(0,100)).zfill(3),"col1":True if random()>0.6 else False,"cnt":randint(10,1500),
                                                    "city":labelList.get(randint(0,15)," ")}).persist()
# because spark is lazy, nothing happens now

In [3]:
myRdd.first()#take(5), collect(), count()
# only now the rdd is created

{'id': '044', 'col1': True, 'cnt': 289, 'city': ' '}

In [4]:
def cleanRows(row):
  keyToClean = []
  for key in row:
    if row[key] == " ":
      keyToClean.append(key)
  for key in keyToClean:
    del row[key]
  return row
    
  
myRdd.map(lambda x: cleanRows(x)).first()

{'id': '044', 'col1': True, 'cnt': 289}

In [5]:
# spark is lazy
testRDD = sc.parallelize(myRdd.take(10)+["this is a bad row"])
testRDD.collect()

[{'id': '044', 'col1': True, 'cnt': 289, 'city': ' '},
 {'id': '066', 'col1': False, 'cnt': 1446, 'city': ' '},
 {'id': '062', 'col1': True, 'cnt': 34, 'city': ' '},
 {'id': '035', 'col1': False, 'cnt': 1468, 'city': ' '},
 {'id': '002', 'col1': False, 'cnt': 705, 'city': ' '},
 {'id': '006', 'col1': False, 'cnt': 753, 'city': ' '},
 {'id': '029', 'col1': False, 'cnt': 1429, 'city': ' '},
 {'id': '055', 'col1': False, 'cnt': 1064, 'city': ' '},
 {'id': '006', 'col1': False, 'cnt': 481, 'city': ' '},
 {'id': '044', 'col1': True, 'cnt': 522, 'city': ' '},
 'this is a bad row']

In [6]:
testRDD.map(lambda x: cleanRows(x)).first()

{'id': '044', 'col1': True, 'cnt': 289}

In [7]:
# if it expects an integer, but you give a string, it returns an error
testRDD.map(lambda x: cleanRows(x)).count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 5.0 failed 1 times, most recent failure: Lost task 7.0 in stage 5.0 (TID 19, 89d9229b3065, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 595, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark/python/pyspark/rdd.py", line 2596, in pipeline_func
    return func(split, prev_func(split, iterator))
  File

In [8]:
# lets try the same but first we discard everything that isn't a dict, and it works
def checkType(row):
  if type(row) == dict:
    return True
  return False
testRDD.filter(lambda x: checkType(x)).map(lambda x: cleanRows(x)).count()

10

In [9]:
# lets another value that is random, and we create 5 rows from each row (1 line = 5 elements)
def extender(row):
  result = []
  for i in range(5):
    newRow = row.copy()
    newRow["likelihood"] = random()
    result.append(newRow)
  return result
extendedMyRdd = myRdd.map(lambda x: extender(x))
extendedMyRdd.first()

[{'id': '044',
  'col1': True,
  'cnt': 289,
  'city': ' ',
  'likelihood': 0.03316169722765783},
 {'id': '044',
  'col1': True,
  'cnt': 289,
  'city': ' ',
  'likelihood': 0.8126952330833833},
 {'id': '044',
  'col1': True,
  'cnt': 289,
  'city': ' ',
  'likelihood': 0.12592211899038785},
 {'id': '044',
  'col1': True,
  'cnt': 289,
  'city': ' ',
  'likelihood': 0.31761329212065825},
 {'id': '044',
  'col1': True,
  'cnt': 289,
  'city': ' ',
  'likelihood': 0.3149683157095011}]

In [10]:
# flatMap allows us to go back to the original shape (1 line 1 element)
extendedMyRdd = myRdd.flatMap(lambda x: extender(x))
extendedMyRdd.first()

{'id': '044',
 'col1': True,
 'cnt': 289,
 'city': ' ',
 'likelihood': 0.1464652077801124}

In [11]:
# lets see the first 5 ordered by likelihood
extendedMyRdd.top(5,key=lambda x: x.get("likelihood"))

[{'id': '074',
  'col1': False,
  'cnt': 321,
  'city': ' ',
  'likelihood': 0.9996761392167998},
 {'id': '097',
  'col1': False,
  'cnt': 1170,
  'city': 'Milano',
  'likelihood': 0.9992988328967947},
 {'id': '086',
  'col1': True,
  'cnt': 1205,
  'city': 'Milano',
  'likelihood': 0.9988334943000686},
 {'id': '056',
  'col1': False,
  'cnt': 676,
  'city': 'Londra',
  'likelihood': 0.9987888416594067},
 {'id': '017',
  'col1': False,
  'cnt': 58,
  'city': ' ',
  'likelihood': 0.9986165433343158}]

In [13]:
# this is an example of distributed mean from lesson 8
# in the reducebykey, x and y are two different contiguous elements (cities) with the same keys (2 lines with same city, because the city is the key), so
# x[0]+y[0] represent the sum of two likelihood from the same cities on 2 different lines. x[1]+y[1] represent the generic sum (how many lines
# with the same cities have been summed)

# (newyork, 222, 440) means that there are 440 lines with newyork as city (key) and the sum of all their likelihoods is 220

#milan X=(X[0]=0.5, X[1]=1)
#milan Y=(Y[0]=0.7, Y[1]=1) 
#                            ==> milan, (1.2,2)
                            #==> milan, (0.3,1) this is the following line with the same key, repeat the process until there are no more lines with the same key (definition of reduce by key)
                                                #==>milan, (1.5,3)
extendedMyRdd.map(lambda  x: (x.get("city"),(x.get("likelihood"),1.0))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).collect()

[('New York', (222.08356349442647, 440.0)),
 ('Roma', (60.95397563863365, 120.0)),
 ('Londra', (188.98926567680272, 360.0)),
 ('Tokyio', (117.28292851348577, 240.0)),
 (' ', (1712.9382286944394, 3400.0)),
 ('Milano', (224.71430443217682, 440.0))]

In [17]:
# this is the same, but we divide the total sum by the total likelihood per each city. then we sort them by the ratio.
extendedMyRdd.filter(lambda x: x.get("city") != " ").map(lambda  x: (x.get("city"),(x.get("likelihood"),1.0)))\
                    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
                    .map(lambda x: (x[0],x[1][0]/x[1][1])).top(10, key=lambda x: x[1])

[('Tokyio', 0.5314478032552876),
 ('New York', 0.5144332403703847),
 ('Londra', 0.499805810888102),
 ('Milano', 0.49326831183665953),
 ('Roma', 0.4455122519031487)]